In [1]:
from mealpy import Problem, FloatVar, BBO, IntegerVar, GA, StringVar
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from mealpy import GA, PSO, Problem, FloatVar, IntegerVar
import numpy as np

SEED = 42


# Our custom problem class
class Squared(Problem):
    def __init__(self, bounds=None, minmax="min", name="Squared", data=None, **kwargs):
        self.name = name
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        x = self.decode_solution(solution)["weight"]
        b = self.decode_solution(solution)["base"]
        return [x**2, b**2]

## Now, we define an algorithm, and pass an instance of our *Squared* class as the problem argument.
# bounds = [IntegerVar(lb=10, ub=100, name="weight"), IntegerVar(lb=10, ub=20, name="base")]
# problem = Squared(bounds=bounds, minmax="min", name="Squared", data="Amazing", obj_weights=(0.5, 0.5))
# model = BBO.OriginalBBO(epoch=100, pop_size=20, seed=SEED)
# model_2 = GA.BaseGA(epoch=100, pop_size=100)
# g_best = model_2.solve(problem)

2024-10-11 00:22:21.113989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 00:22:26.233245: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 00:22:28.539681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 00:22:33.610328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 00:22:34.925395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 00:22:42.547371: I tensorflow/core/platform/cpu_feature_gu

In [2]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [69]:
class LogisticRegression():
    def __init__(self, input: int, optimizer: str, learning_rate: float):
        self.input = input
        self.optimizer_name = optimizer
        self.learning_rate = learning_rate
        self.sequential = Sequential([
            Input(shape=(input,)),
            Dense(1, activation='linear'),
        ])

    def fit(self, X, y, epochs=50):
        self.sequential.compile(
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=getattr(tf.keras.optimizers, self.optimizer_name)(learning_rate=self.learning_rate))
        history = self.sequential.fit(X, y, epochs=epochs, verbose=0)
        last_loss = history.history["loss"][-1]
        return last_loss



In [60]:
model = LogisticRegression(X_train.shape[1], optimizer="Adam", learning_rate=1e-3)

In [63]:
a = model.fit(X_train, y_train, epochs=100)

Epoch 1/100


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 0.3086
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 0.3228
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 0.2522
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 0.3038
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 0.2704
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 0.3332
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - loss: 0.3065
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 0.3057
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - loss: 0.3188
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 0.4131
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4267 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - loss: 0.3977
Epoch 13/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - loss: 0.3207
Epoch 14/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3263
Epoch 15/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - lo

In [62]:
a

0.3351092040538788

In [64]:
model.sequential.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1025  


0.11001080274581909

In [70]:
class HyperParamProblem(Problem):
    def __init__(self, bounds, minmax="min", data=None, **kwargs):
        self.data = data

        super().__init__(bounds, minmax, **kwargs)

    def train_evaluate_model(self, optimizer_name, learning_rate):
        input_dim = self.data["X_train"].shape[1]
        model = LogisticRegression(input=input_dim, optimizer=optimizer_name, learning_rate=learning_rate)
        loss_train = model.fit(self.data["X_train"], self.data["y_train"], epochs=self.data["epoch"])
        loss_test = model.sequential.evaluate(self.data["X_test"], self.data["y_test"], verbose=0)

        return [loss_train, loss_test]

    def obj_func(self, solution):
        solution = self.decode_solution(solution)
        optimizer_name, learning_rate = solution["optimizer"], solution["learning_rate"]
        train_test_loss = self.train_evaluate_model(optimizer_name, learning_rate)
        return train_test_loss

In [71]:
data = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "epoch": 100
}
optimizers = [ "SGD", "Adam", "RMSprop", "Adagrad", "Adadelta", "Adamax"]
bounds = [
    StringVar(valid_sets=optimizers, name="optimizer"),
    FloatVar(lb=0, ub=0.1, name="learning_rate"),
]

In [72]:
hyperparam_problem = HyperParamProblem(bounds=bounds, minmax="min", data=data, name="Hyperparam_problem", obj_weights=(0.5, 0.5))

In [75]:
model = PSO.OriginalPSO(epoch=100, pop_size=20, seed=SEED)
model_2 = GA.BaseGA(epoch=5, pop_size=50)
g_best = model_2.solve(hyperparam_problem, mode='thread')

2024/10/11 01:05:25 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving 2-objective optimization problem with weights: [0.5 0.5].
2024/10/11 01:05:25 AM, WARNING, mealpy.evolutionary_based.GA.BaseGA: The parallel mode: thread is selected. But n_workers is not set. The default n_workers = 4 is used.
2024/10/11 01:09:27 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 1, Current best: 0.11007980443537235, Global best: 0.11007980443537235, Runtime: 121.94585 seconds
2024/10/11 01:11:28 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 2, Current best: 0.10327091440558434, Global best: 0.10327091440558434, Runtime: 121.11764 seconds


2024/10/11 01:13:34 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 3, Current best: 0.09247362054884434, Global best: 0.09247362054884434, Runtime: 125.24445 seconds
2024/10/11 01:15:39 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 4, Current best: 0.09684869460761547, Global best: 0.09247362054884434, Runtime: 125.02938 seconds
2024/10/11 01:17:41 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 5, Current best: 0.10671620070934296, Global best: 0.09247362054884434, Runtime: 122.50920 seconds


In [76]:
print(f"Best agent: {g_best}")
print(f"Best solution: {g_best.solution}")
print(f"Best accuracy: {g_best.target.fitness}")
print(f"Best parameters: {model_2.problem.decode_solution(g_best.solution)}")

Best agent: id: 1125, target: Objectives: [0.12693752 0.05800972], Fitness: 0.09247362054884434, solution: [3.        0.0238843]
Best solution: [3.        0.0238843]
Best accuracy: 0.09247362054884434
Best parameters: {'optimizer': 'Adamax', 'learning_rate': 0.023884296520883076}


In [ ]:
g_best_pso = model.solve(problem=Problem, mode="thread")